## Collect data and Create sample

In [ ]:
import bonsai_ipcc

my_ppf = bonsai_ipcc.PPF()


import pandas as pd
import numpy as np

df_coke = pd.DataFrame(
        data={
            "year": [2016, 2016, 2016, 2016, 2016],
            "region": ["CN", "CN", "CN", "CN", "CN"],
            "property": ["def", "min", "max", "abs_min", "abs_max"],
            "value": [449114810.0, 404203329.0, 494026291.0, 0, np.inf],#https://data.un.org/Data.aspx?q=coke&d=ICS&f=cmID%3a33100-0 + 10% uncertainty https://www.ipcc-nggip.iges.or.jp/public/2019rf/pdf/3_Volume3/19R_V3_Ch04_Metal_Industry.pdf
            "unit": ["t/yr", "t/yr", "t/yr", "t/yr", "t/yr"],
        }
).set_index(["year", "region", "property"])

my_ppf.ppf_vol.metal.parameter.coke=df_coke

df_coke_act = pd.DataFrame(
        data={
            "year": [2016, 2016, 2016, 2016, 2016,2016, 2016, 2016, 2016, 2016],
            "region": ["CN", "CN", "CN", "CN", "CN","CN", "CN", "CN", "CN", "CN"],
            "activity": ["by-product_recovery","by-product_recovery","by-product_recovery","by-product_recovery","by-product_recovery",
                         "no_by-product_recovery","no_by-product_recovery","no_by-product_recovery","no_by-product_recovery","no_by-product_recovery"],
            "property": ["def", "min", "max", "abs_min", "abs_max","def", "min", "max", "abs_min", "abs_max"],
            "value": [0.1, 0.0, 0.2, 0, 1, 0.9,0.7,1.0,0,1],
            "unit": ["t/t", "t/t", "t/t", "t/t", "t/t","t/t", "t/t", "t/t", "t/t", "t/t"],
        }
).set_index(["year", "region", "activity","property"])

my_ppf.ppf_vol.metal.parameter.coke_activity_per_reg=df_coke_act

#year,region,activity,property,value,unit
#2019,World,by-product_recovery,def,0.51,t/t
#2019,World,by-product_recovery,min,0.459,t/t
#2019,World,by-product_recovery,max,0.561,t/t
#2019,World,by-product_recovery,abs_min,0.0,t/t
#2019,World,by-product_recovery,abs_max,inf,t/t
#2019,World,no_by-product_recovery,def,1.23,t/t
#2019,World,no_by-product_recovery,min,1.107,t/t
#2019,World,no_by-product_recovery,max,1.353,t/t
#2019,World,no_by-product_recovery,abs_min,0.0,t/t
#2019,World,no_by-product_recovery,abs_max,inf,t/t



emission_transf_coeff_coke = my_ppf.ppf_vol.metal.parameter.emission_transf_coeff_coke
feedstock_use_per_coke = my_ppf.ppf_vol.metal.parameter.feedstock_use_per_coke
energy_use_per_coke = my_ppf.ppf_vol.metal.parameter.energy_use_per_coke
coke_activity_per_reg = my_ppf.ppf_vol.metal.parameter.coke_activity_per_reg
product_transf_coeff_coke = my_ppf.ppf_vol.metal.parameter.product_transf_coeff_coke
coal_use_per_coke = my_ppf.ppf_vol.metal.parameter.coal_use_per_coke
byproduct_supply_per_coke = my_ppf.ppf_vol.metal.parameter.byproduct_supply_per_coke
emission_per_coke = my_ppf.ppf_vol.metal.parameter.emission_per_coke
carbon_content_per_mass = my_ppf.ppf_vol.metal.parameter.carbon_content_per_mass

dict_of_dfs = {
    "emission_transf_coeff_coke":emission_transf_coeff_coke,
    "feedstock_use_per_coke": feedstock_use_per_coke,
    "energy_use_per_coke":energy_use_per_coke,
    "coke_activity_per_reg":coke_activity_per_reg,
    "product_transf_coeff_coke":product_transf_coeff_coke,
    "coal_use_per_coke":coal_use_per_coke,
    "byproduct_supply_per_coke":byproduct_supply_per_coke,
    "emission_per_coke":emission_per_coke,
    "coke":df_coke,
    "carbon_content_per_mass":carbon_content_per_mass,
    "coke_activity_per_reg":df_coke_act,
}

constraints={"coke_activity_per_reg":{"group_by":["year", "region"]}}

sampled_dfs=[]
for k,v in dict_of_dfs.items():
    if k == "coke_activity_per_reg":
        cons = constraints
        r=my_ppf.create_sample(dfs={f"{k}":v},constraints=cons)
    else:
        cons=None
        r=my_ppf.create_sample(dfs={f"{k}":v},constraints=cons)
    sampled_dfs.append(r)

for s in sampled_dfs:
    for k,v in s.items():
        setattr(my_ppf.ppf_vol.metal.parameter,k,v)




## Run sequence and calculate Use, Emission and Supply tables 

In [ ]:
s_byprod_sample = my_ppf.ppf_vol.metal.sequence.coke_tier1(year=2016,region="CN",activity="by-product_recovery",uncertainty="sample")
s_no_byprod_sample = my_ppf.ppf_vol.metal.sequence.coke_tier1(year=2016,region="CN",activity="no_by-product_recovery",uncertainty="sample")

In [ ]:
dfs=s_byprod_sample.to_frames(bonsai="sample")

In [ ]:
dfs_byprod_samples=s_byprod_sample.to_frames(bonsai="samples")
dfs_no_byprod_samples=s_no_byprod_sample.to_frames(bonsai="samples")

dfs_byprod_unc=s_byprod_sample.to_frames(bonsai=True)
dfs_no_byprod_unc=s_no_byprod_sample.to_frames(bonsai=True)

In [ ]:
use=pd.concat([dfs_byprod_samples["bonsai"]["use"],dfs_no_byprod_samples["bonsai"]["use"]]).reset_index(drop=True)
use

In [ ]:
supply=pd.concat([dfs_byprod_samples["bonsai"]["supply"],dfs_no_byprod_samples["bonsai"]["supply"]]).reset_index(drop=True)

supply["product"] = supply["product"].replace("coke_from_by-product_recovery", "coke")
supply["product"] = supply["product"].replace("coke_from_no_by-product_recovery", "coke")
supply

In [ ]:
emission=pd.concat([dfs_byprod_samples["bonsai"]["emission"],dfs_no_byprod_samples["bonsai"]["emission"]]).reset_index(drop=True)
emission

In [8]:
transf_coeff=pd.concat([sampled_dfs[4]["product_transf_coeff_coke"], sampled_dfs[0]["emission_transf_coeff_coke"]])
transf_coeff=transf_coeff.loc[:,:,:,:,:,"sample"]

In [9]:
# specify products those include Carbon
list_of_values=["coke", "tar", "coke_oven_gas", "sulphur", "benzene"]


In [ ]:
df=emission[["activity","emission_substance","samples","unit"]]
df=df.rename(columns={"emission_substance":"product"})
df

In [ ]:
df_s=supply[supply['product'].isin(list_of_values)]
df_s=df_s[["activity","product","samples","unit"]]
df_s

In [ ]:
df_u=use[use["product"]=="coking_coal"]
df_u=df_u[["activity","product","samples","unit"]]
df_u

In [13]:
df_s["flow_type"]="output"
df_u["flow_type"]="input"
df["flow_type"]="output"

In [ ]:
df_total=pd.concat([df,df_s,df_u])
df_total

In [15]:
df_total_adj=df_total.copy()

## Reconcile data to fulfill carbon balance

In [ ]:
carbon_df=sampled_dfs[9]["carbon_content_per_mass"].reset_index()


carbon_df["coefficient"]=carbon_df["value"].apply(lambda x: x.mean() if isinstance(x, np.ndarray) else x)


carbon_df["std_dev"]=carbon_df["value"].apply(lambda x: x.std() if isinstance(x, np.ndarray) else 0.0)#0001)
carbon_df

In [ ]:
import numpy as np

carbon_df = sampled_dfs[9]["carbon_content_per_mass"].reset_index()

carbon_df["coefficient"] = carbon_df["value"].apply(lambda x: x.mean() if isinstance(x, np.ndarray) else x)
carbon_df["std_dev"] = carbon_df["value"].apply(lambda x: x.std() if isinstance(x, np.ndarray) else 0.001)

# Remove rows where "value" is exactly 0.0 or an array of only zeros
carbon_df = carbon_df.loc[~carbon_df["value"].apply(lambda x: np.all(x == 0.0) if isinstance(x, (np.ndarray, list)) else x == 0.0)]

# Drop columns where all values are exactly 0.0
carbon_df = carbon_df.drop(columns=[
    col for col in carbon_df.columns 
    if carbon_df[col].apply(lambda x: np.all(x == 0.0) if isinstance(x, (np.ndarray, list)) else x == 0.0).all()
])

carbon_df




In [18]:
conversion_df=transf_coeff.reset_index()[["activity","product","reference_output","value"]]
conversion_df=conversion_df.rename(columns={"value":"value", "product": "input_product","reference_output":"output_product"})
conversion_df["coefficient"]=conversion_df["value"].apply(lambda x: x.mean())
conversion_df["std_dev"]=conversion_df["value"].apply(lambda x: x.std())

In [19]:

df_total_adj=df_total.loc[df_total["product"] !="sulphur"]

In [20]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

def rescale_least_square(activity, df_total_adj, conversion_df=None, carbon_df=None):
    # Filter data for the specified activity
    df = df_total_adj[df_total_adj["activity"] == activity].copy()

    # Extract means and standard deviations of samples for each product
    df['mean'] = df['samples'].apply(np.mean)
    df['std_dev'] = df['samples'].apply(np.std)

    # Identify unique products and map them to indices
    unique_products = df['product'].unique()
    product_to_index = {product: i for i, product in enumerate(unique_products)}

    # Extract conversion coefficients if provided
    if conversion_df is not None:
        conversion_df = conversion_df[conversion_df["activity"] == activity].copy()
        coefficients = conversion_df['coefficient'].values
        coefficient_std = conversion_df['std_dev'].values
    else:
        coefficients = np.array([])  # Empty array to avoid indexing issues
        coefficient_std = np.array([])

    # Extract carbon content if provided
    if carbon_df is not None:
        carbon_df = carbon_df[carbon_df["product"].isin(unique_products)].copy()
        carbon_content_dict = dict(zip(carbon_df['product'], carbon_df['coefficient']))
        carbon_std_dict = dict(zip(carbon_df['product'], carbon_df['std_dev']))
    else:
        carbon_content_dict = {}
        carbon_std_dict = {}

    # Function to build conversion constraints
    def build_conversion_constraints():
        if conversion_df is None:
            return []

        constraints = []
        for coeff_index, row in enumerate(conversion_df.itertuples(index=False)):
            input_index = product_to_index[row.input_product]
            output_index = product_to_index[row.output_product]

            def constraint(x, input_index=input_index, output_index=output_index, coeff_index=coeff_index):
                product_values = x[:len(unique_products)]
                dynamic_coeff = x[len(unique_products) + coeff_index]  # Optimized coefficient
                return product_values[input_index] * dynamic_coeff - product_values[output_index]

            constraints.append({'type': 'eq', 'fun': constraint})

        return constraints

    # Carbon balance constraint
    def carbon_balance_constraint(x):
        if carbon_df is None:
            return 0  # No constraint applied

        num_products = len(unique_products)
        num_coefficients = len(coefficients)

        product_values = x[:num_products]  # Optimized product values
        carbon_values = x[num_products + num_coefficients:]  # Optimized carbon content

        total_carbon_in = sum(
            product_values[i] * carbon_values[i]
            for i, product in enumerate(unique_products)
            if product in df_total_adj[df_total_adj["flow_type"] == "input"]["product"].values
        )

        total_carbon_out = sum(
            product_values[i] * carbon_values[i]
            for i, product in enumerate(unique_products)
            if product in df_total_adj[df_total_adj["flow_type"] == "output"]["product"].values
        )

        return total_carbon_in - total_carbon_out

    # Objective function: minimize residuals and penalties
    def objective(x):
        num_products = len(unique_products)
        product_values = x[:num_products]
        dynamic_coefficients = x[num_products:num_products + len(coefficients)] if conversion_df is not None else []
        carbon_values = x[num_products + len(coefficients):] if carbon_df is not None else []

        # Residuals for product values
        residuals = sum(
            ((row['mean'] - product_values[product_to_index[row['product']]]) / row['std_dev']) ** 2
            for _, row in df.iterrows()
        )

        # Penalty for coefficient deviations
        coefficient_penalty = (
            np.sum(((dynamic_coefficients - coefficients) / coefficient_std) ** 2) 
            if conversion_df is not None and len(coefficients) > 0 else 0
        )

        # Penalty for carbon content deviations
        carbon_penalty = 0
        if carbon_df is not None:
            for i, product in enumerate(unique_products):
                observed_carbon = carbon_content_dict.get(product, 0)
                std_dev = carbon_std_dict.get(product, 0.1)  # Avoid division by zero
                carbon_penalty += ((carbon_values[i] - observed_carbon) / std_dev) ** 2

        return residuals + coefficient_penalty + carbon_penalty

    # Setup optimization variables
    num_products = len(unique_products)
    num_coefficients = len(coefficients)
    num_carbon_values = num_products if carbon_df is not None else 0

    # Set up constraints
    constraints = []
    if conversion_df is not None:
        constraints.extend(build_conversion_constraints())

    if carbon_df is not None:
        constraints.append({'type': 'eq', 'fun': carbon_balance_constraint})

    # Initial guess: product means, nominal coefficients, and carbon content
    initial_guess = np.concatenate([
        df.groupby('product')['mean'].mean().reindex(unique_products).fillna(1).values,  # Product values
        coefficients if conversion_df is not None else np.array([]),  # Conversion coefficients
        np.array([carbon_content_dict.get(p, 1) for p in unique_products]) if carbon_df is not None else np.array([])  # Carbon content
    ])

    # Solve the optimization problem
    result = minimize(
        fun=objective,
        x0=initial_guess,
        constraints=constraints,
        tol=1e-6,
        method='SLSQP',
        options={'disp': True, 'maxiter': 1000}
    )

    # Process results
    if result.success:
        print("Optimization successful!")
        optimized_values = result.x[:num_products]
        optimized_coefficients = result.x[num_products:num_products + num_coefficients] if conversion_df is not None else []
        optimized_carbon_values = result.x[num_products + num_coefficients:] if carbon_df is not None else []

        # Prepare results
        product_data = [{"product": product, "value": value} for product, value in zip(unique_products, optimized_values)]
        result_df_products = pd.DataFrame(product_data)

        result_df_coefficients = None
        if conversion_df is not None:
            coefficient_data = [
                {"input_product": row['input_product'], "output_product": row['output_product'], "optimized_coefficient": coeff}
                for row, coeff in zip(conversion_df.to_dict(orient='records'), optimized_coefficients)
            ]
            result_df_coefficients = pd.DataFrame(coefficient_data)

        result_df_carbon = None
        if carbon_df is not None:
            carbon_data = [{"product": product, "optimized_carbon_content": value} for product, value in zip(unique_products, optimized_carbon_values)]
            result_df_carbon = pd.DataFrame(carbon_data)

        return result_df_products, result_df_coefficients, result_df_carbon
    else:
        print("Optimization failed!")
        print(result.message)
        return None, None, None


In [ ]:
result_df, result_coeff, result_carbon = rescale_least_square("by-product_recovery", df_total_adj, conversion_df=None, carbon_df=carbon_df)#carbon_df)

In [ ]:
result_df_2, result_coeff_2,result_carbon_2= rescale_least_square("no_by-product_recovery", df_total_adj, conversion_df=None, carbon_df=carbon_df)#carbon_df)


## Plot results

In [23]:
# add activity
result_df["activity"]="by-product_recovery"


In [24]:
# add activity
result_df_2["activity"]="no_by-product_recovery"


In [ ]:

reconciled_df = pd.concat([result_df,result_df_2])
reconciled_df

### Plot Use

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
# Unique products and activities
products = use["product"].unique()
activities = use["activity"].unique()

# Create subplots
n_products = len(products)
fig, axes = plt.subplots(nrows=(n_products + 1) // 2, ncols=2, figsize=(12, 10), constrained_layout=True)
axes = axes.flatten()  # Flatten for easier indexing

# Store legend elements
handles_labels = {}

# Iterate over products and plot
for idx, product in enumerate(products):
    ax = axes[idx]
    product_data = use[use["product"] == product]
    
    for activity in activities:
        activity_data = product_data[product_data["activity"] == activity]
        samples = activity_data["samples"].sum()
        mean_value = activity_data["samples"].apply(lambda x: sum(x) / len(x)).mean()
        
        hist = ax.hist(samples, label=f"{activity}", alpha=0.7, bins=100)
        color = 'red' if activity == "no_by-product_recovery" else 'blue'
        linestyle = '--'
        
        mean_line = ax.axvline(mean_value, color=color, linestyle=linestyle, linewidth=1.5, label="mean")
        recon_line = None
        if product == "coking_coal":
            try:
                reconciled_value = reconciled_df[(reconciled_df["activity"] == activity) & (reconciled_df["product"] == product)]["value"].iloc[0]
                recon_line = ax.axvline(reconciled_value, color=color, linestyle='-', linewidth=1.5, label="recon")
            except:
                pass
        
        handles_labels[activity] = (hist[2][0], f"{activity}")
        handles_labels[f"{activity}_mean"] = (mean_line, "mean")
        if recon_line:
            handles_labels[f"{activity}_recon"] = (recon_line, "recon")
    
    unit = use.loc[idx]["unit"]
    ax.set_title(f"Use: {product}")
    ax.set_xlabel(f"{unit}")
    ax.set_ylabel("Frequency")

# Remove unused subplots
for i in range(len(products), len(axes)):
    fig.delaxes(axes[i])

# Create a single legend in one row
handles, labels = zip(*handles_labels.values())
fig.legend(handles, labels, loc="upper center", bbox_to_anchor=(0.5, -0.05), ncol=len(labels))

plt.show()

### Plot Supply and Emission

In [ ]:
# Unique products and activities
products = supply["product"].unique()
activities = supply["activity"].unique()
products_e = emission["emission_substance"].unique()

# Create subplots
n_products = len(products) + len(products_e)
fig, axes = plt.subplots(nrows=(n_products + 1) // 2, ncols=2, figsize=(12, 10), constrained_layout=True)
axes = axes.flatten()  # Flatten for easier indexing

# Store legend elements
handles_labels = {}

# Iterate over products and plot
for idx, product in enumerate(products):
    ax = axes[idx]
    product_data = supply[supply["product"] == product]
    
    for activity in activities:
        activity_data = product_data[product_data["activity"] == activity]
        samples = activity_data["samples"]
        mean_value = activity_data["samples"].apply(lambda x: sum(x) / len(x)).mean()
        
        hist = ax.hist(samples, label=f"{activity}", alpha=0.7, bins=100)
        color = 'red' if activity == "no_by-product_recovery" else 'blue'
        linestyle = '--'
        
        mean_line = ax.axvline(mean_value, color=color, linestyle=linestyle, linewidth=1.5, label="mean")
        try:
            reconciled_value = reconciled_df[(reconciled_df["activity"] == activity) & (reconciled_df["product"] == product)]["value"].iloc[0]
            recon_line = ax.axvline(reconciled_value, color=color, linestyle='-', linewidth=1.5, label="recon")
        except:
            recon_line = None
        
        handles_labels[activity] = (hist[2][0], f"{activity}")
        handles_labels[f"{activity}_mean"] = (mean_line, "mean")
        if recon_line:
            handles_labels[f"{activity}_recon"] = (recon_line, "reconciled")
    
    unit = supply.loc[idx]["unit"]
    ax.set_title(f"Supply: {product}")
    ax.set_xlabel(f"{unit}")
    ax.set_ylabel("Frequency")

# Iterate over emission products and plot
for idx, product in enumerate(products_e):
    ax = axes[idx + len(products)]
    product_data = emission[emission["emission_substance"] == product]
    
    for activity in activities:
        activity_data = product_data[product_data["activity"] == activity]
        samples = activity_data["samples"]
        mean_value = activity_data["samples"].apply(lambda x: sum(x) / len(x)).mean()
        
        hist = ax.hist(samples, label=f"{activity}", alpha=0.7, bins=100)
        color = 'red' if activity == "no_by-product_recovery" else 'blue'
        linestyle = '--'
        
        mean_line = ax.axvline(mean_value, color=color, linestyle=linestyle, linewidth=1.5, label="mean")
        try:
            reconciled_value = reconciled_df[(reconciled_df["activity"] == activity) & (reconciled_df["product"] == product)]["value"].iloc[0]
            recon_line = ax.axvline(reconciled_value, color=color, linestyle='-', linewidth=1.5, label="reconciled")
        except:
            recon_line = None
        
        handles_labels[activity] = (hist[2][0], f"{activity}")
        handles_labels[f"{activity}_mean"] = (mean_line, "mean")
        if recon_line:
            handles_labels[f"{activity}_recon"] = (recon_line, "reconciled")
    
    unit = emission.loc[idx]["unit"]
    ax.set_title(f"Emission: {product}")
    ax.set_xlabel(f"{unit}")
    ax.set_ylabel("Frequency")

# Remove unused subplots
for i in range(n_products, len(axes)):
    fig.delaxes(axes[i])

# Create a single legend
handles, labels = zip(*handles_labels.values())
fig.legend(handles, labels, loc="upper center", bbox_to_anchor=(0.5, -0.05), ncol=len(labels))

plt.show()


In [28]:
result_carbon_2["activity"]="no_by-product_recovery"
result_carbon["activity"]="by-product_recovery"

In [ ]:
carb_df=pd.concat([result_carbon,result_carbon_2])
carb_df

In [30]:
result_carbon_2["activity"]="no_by-product_recovery"
result_carbon["activity"]="by-product_recovery"

In [ ]:
carb_df=pd.concat([result_carbon,result_carbon_2])
carb_df

In [ ]:
carbon_df[(carbon_df["product"] == "coke")]

### Plot carbon content factors

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math


B=carb_df.copy()
A=carbon_df.copy()

# Get unique products
products = B["product"].unique()
n_products = len(products)

# Set up subplots in 2 columns
ncols = 3
nrows = math.ceil(n_products / ncols)  # Calculate number of rows needed
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 2 * nrows), sharey=True)

# Flatten axes array for easy iteration
axes = axes.flatten()

# Iterate over products and axes
for i, (product, ax) in enumerate(zip(products, axes)):
    # Filter data for the product
    old_mean = A[(A["product"] == product) & (A["property"] == "sample")]["coefficient"].values[0]
    old_std_dev = A[(A["product"] == product) & (A["property"] == "sample")]["std_dev"].values[0]

    ax.axvline(old_mean, color="black", linestyle="solid", linewidth=2, label="old_mean")
    
    perc_2_5= old_mean-1.96*old_std_dev
    per_97_5= old_mean+1.96*old_std_dev

    ax.axvline(perc_2_5, color="black", linestyle="dashed", linewidth=2, label="per_2_5")
    ax.axvline(per_97_5, color="black", linestyle="dashed", linewidth=2, label="per_97_5")


    recon_low = B[(B["product"] == product) & (B["activity"] == "by-product_recovery")]["optimized_carbon_content"].values[0]
    recon_high = B[(B["product"] == product) & (B["activity"] == "no_by-product_recovery")]["optimized_carbon_content"].values[0]

    ax.axvline(recon_low, color="red", linestyle="solid", linewidth=2, label="by-product_recovery")
    ax.axvline(recon_high, color="green", linestyle="solid", linewidth=2, label="no_by-product_recovery")


    ax.set_title(f"Product: {product}")
    ax.set_xlabel("Value")
    ax.set_yticks([])

# Hide unused subplots
for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

## Check carbon balances

In [ ]:
df_merged=carb_df.merge(reconciled_df)
df_merged

In [ ]:
df_merged["t C"]=df_merged["value"]*df_merged["optimized_carbon_content"]
df_merged

In [ ]:
# c input (by-product_recovery)
df_merged[(df_merged["activity"]=="by-product_recovery") & (df_merged["product"]=="coking_coal")]["t C"][6]

In [ ]:
# c outout (by-product_recovery)
df_merged[(df_merged["activity"]=="by-product_recovery") & (df_merged["product"]!="coking_coal")]["t C"].sum()

In [ ]:
# c input (no_by-product_recovery)
df_merged[(df_merged["activity"]=="no_by-product_recovery") & (df_merged["product"]=="coking_coal")]["t C"][13]

In [ ]:
# c outout (no_by-product_recovery)
df_merged[(df_merged["activity"]=="no_by-product_recovery") & (df_merged["product"]!="coking_coal")]["t C"].sum()

In [ ]:
emission_per_coke